In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

import json 

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Data Preprocessing and Data Exploration

Possible Analysis (at a later stage): Use the Jaccard Similarity to check whether there is difference in the top 10 to 20 popular genres and tags between dramas from South Korea, Thailand, and Japan

Fighting against the formitable opponent know as Korean Drama, Lakorn Thai and Japanese Drama cannot compete in terms of overall viewership. We argued that Thai and Japanese drama are niche categories of drama that are noteworthy of being classified as "Hidden Gem of Asia" - they are niche, but they are outstanding.

Given the context and the theory we proposed, we argued that it is not appropriate to use the total viewership to make judgement regarding the drama performance. Hence, we came up with a metric to help with the comparison. 

Metric: `scored_significant = total user score * total number of user that casted the vote / total people who watched the drama`

In [ ]:
def split_data(df: pd.DataFrame) -> pd.DataFrame:
    df['genres'] = df['genres'].apply(lambda x: [char.replace(',','').strip() for char in str(x).split(',')])
    df['tags'] = df['tags'].apply(lambda x: [char.replace(',','').strip()  for char in str(x).split(',')])
    df['aired_on'] = df['aired_on'].apply(lambda x: [char.replace(',','').strip()  for char in str(x).split()])
    return df

# Test 
def split_data_test(df: pd.DataFrame) -> pd.DataFrame:
    df['genres'] = df['genres'].apply(lambda x: [char.strip() for char in str(x).split(',')])
    df['tags'] = df['tags'].apply(lambda x: [char.strip() for char in str(x).split(',')])
    df['aired_on'] = df['aired_on'].apply(lambda x: [char.strip() for char in str(x).split(',')])
    return df

def custom_rating_generator(df: pd.DataFrame) -> pd.DataFrame:
    df['watched_ratio'] = (df['tot_num_user'] / df['tot_watched']).round(3)
    df['scored_signif'] = df['tot_user_score'] * df['watched_ratio']
    return df

def fillna_numerical_columns(df: pd.DataFrame) -> pd.DataFrame:
    numerical_columns = df.describe().columns.tolist()
    
    for col in numerical_columns:
        df[col] = df[col].fillna(0.0)
        
    return df

def drama_process_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    
    # Filtered for prefered data
    df['year'] = df['year'].apply(lambda x: str(x))
    
    df = df[
        (df['type'].isin(['Drama','Movie'])) & 
        (df['year'].isin(['2021','2022','2023']))
    ]
    
    # df = split_data(df)
    df = split_data_test(df)
    df = custom_rating_generator(df)
    df = fillna_numerical_columns(df)
    
    removed_features = [
        'drama_id','synopsis','rank','popularity',
        'director','sc_writer','start_dt', 'end_dt'
    ]
    df = df.drop(columns=removed_features,axis=1)
    
    return df

In [ ]:
test = pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/tha_drama.csv')
test.loc[:.1]['tags'][0].split(',')

In [ ]:
# Preprocess Drama Metadata Dataset
dtha_df = drama_process_pipeline(pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/tha_drama.csv'))
dkor_df = drama_process_pipeline(pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/kor_drama.csv'))
djap_df = drama_process_pipeline(pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/jap_drama.csv'))

# Extract Drama Name
# tha_dname = dtha_df[dtha_df['type'] == 'Drama']['drama_name'].tolist()
# kor_dname = dkor_df[dkor_df['type'] == 'Drama']['drama_name'].tolist()
# jap_dname = djap_df[djap_df['type'] == 'Drama']['drama_name'].tolist()

# print(f'Thai: {dtha_df.shape[0]}, #drama := {len(tha_dname)}')
# print(f'South Korea: {dkor_df.shape[0]}, #drama := {len(kor_dname)}')
# print(f'Japan: {djap_df.shape[0]}, #drama := {len(jap_dname)}')

# # Valid Drama Dicts should be placed after the preprocessing task (remove outliers etc. )
# global valid_drama_dict 

# valid_drama_dict = {
#     'Thailand': tha_dname,
#     'South Korea': kor_dname,
#     'Japan': jap_dname
# }

# # Load the data: ignoring drama_id, rank, and pop(ularity) because we won't be using the website ranking system
# dtha_df = pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/tha_drama.csv').iloc[:,1:-2]
# dkor_df = pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/kor_drama.csv').iloc[:,1:-2]
# djap_df = pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/jap_drama.csv').iloc[:,1:-2]


# # Extract Drama Name
# tha_dname = dtha_df[dtha_df['type'] == 'Drama']['drama_name'].tolist()
# kor_dname = dkor_df[dkor_df['type'] == 'Drama']['drama_name'].tolist()
# jap_dname = djap_df[djap_df['type'] == 'Drama']['drama_name'].tolist()

In [ ]:
# Filter for only drama
dtha_df = dtha_df[dtha_df['type'] == 'Drama']
dkor_df = dkor_df[dkor_df['type'] == 'Drama']
djap_df = djap_df[djap_df['type'] == 'Drama']

print(f'Thai: {dtha_df.shape[0]}')
print(f'South Korea: {dkor_df.shape[0]}')
print(f'Japan: {djap_df.shape[0]}')

Removing drama that falls below the 25th percentile of viewership since we want to focus on more popular or the better representative drams of each region. We can consider the alternative where we collected the data with overall viewership greater than the average viewship. However, this approach will make the remaining dataset really small.

In [ ]:
'''PROBABLY DONT NEED THIS FUNCTION GIVEN THE CURRENT ANALYSIS'''
def process_percentile_watched_pipeline(
    data_frame: pd.DataFrame,
    column_name: str,
    percentile: float = 0.25,
) -> pd.DataFrame:
    
    lower_percentile = data_frame[column_name].quantile(percentile) 
    
    filtered_data = data_frame[data_frame[column_name] >= lower_percentile]
    
    removed_data = data_frame[data_frame[column_name]< lower_percentile]
    
    print(f"Output report:\nFiltered df: {filtered_data.shape[0]}\nRemoved: {removed_data.shape[0]}")
    return filtered_data, removed_data

In [ ]:
def process_outliers_df_pipeline(
    data_frame: pd.DataFrame,
    column_name: str,
    iqr_multiplier: float = 1.5
) -> pd.DataFrame:
    
    q1 = data_frame[column_name].quantile(0.25)
    q3 = data_frame[column_name].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - iqr_multiplier * iqr
    upper_bound = q3 + iqr_multiplier * iqr

    outliers_df = data_frame[(data_frame[column_name] < lower_bound) | (data_frame[column_name] > upper_bound)]
    filtered_df = data_frame[(data_frame[column_name] >= lower_bound) & (data_frame[column_name] <= upper_bound)]
    
    print(f"Pipeline Report:\nOriginal df: {data_frame.shape[0]}\nOutliers df: {outliers_df.shape[0]}\nCleaned df: {filtered_df.shape[0]}",end="\n\n")
    
    return outliers_df, filtered_df

In [ ]:
# Preprocess the data (removed outliers)
print("Thailand:")
dtha_df_outliers, dtha_df_filtered = process_outliers_df_pipeline(dtha_df, 'scored_signif', 1.5)
print("Korea:")
dkor_df_outliers, dkor_df_filtered = process_outliers_df_pipeline(dkor_df, 'scored_signif', 1.5)
print("Japan:")
djap_df_outliers, djap_df_filtered = process_outliers_df_pipeline(djap_df, 'scored_signif', 1.5)

In [ ]:
tha_dname = dtha_df_filtered[dtha_df_filtered['type'] == 'Drama']['drama_name'].tolist()
kor_dname = dkor_df_filtered[dkor_df_filtered['type'] == 'Drama']['drama_name'].tolist()
jap_dname = djap_df_filtered[djap_df_filtered['type'] == 'Drama']['drama_name'].tolist()

print(f'Thai: {dtha_df.shape[0]}, #valid drama := {len(tha_dname)}')
print(f'South Korea: {dkor_df.shape[0]}, #valid drama := {len(kor_dname)}')
print(f'Japan: {djap_df.shape[0]}, #valid drama := {len(jap_dname)}')

# Valid Drama Dicts should be placed after the preprocessing task (remove outliers etc. )
global valid_drama_dict 

valid_drama_dict = {
    'Thailand': tha_dname,
    'South Korea': kor_dname,
    'Japan': jap_dname
}

In [ ]:
dkor_df_filtered['above_mean'] = np.where(
    dkor_df_filtered['scored_signif'] >= dkor_df_filtered['scored_signif'].mean(),
    'High','Low'
)

dtha_df_filtered['above_mean'] = np.where(
    dtha_df_filtered['scored_signif'] >= dtha_df_filtered['scored_signif'].mean(),
    'High','Low'
)

djap_df_filtered['above_mean'] = np.where(
    djap_df_filtered['scored_signif'] >= djap_df_filtered['scored_signif'].mean(),
    'High','Low'
)

combined_drama_df = pd.concat([dkor_df_filtered, dtha_df_filtered, djap_df_filtered], axis=0, ignore_index=True)

combined_outliers_df = pd.concat([dkor_df_outliers, dtha_df_outliers, djap_df_outliers], axis=0, ignore_index=True)

In [ ]:
combined_drama_df.sample(n=3)

In [ ]:
# Beepswarm plot to show the distribution of Score Significant 
# The color shows the seperation of data where the red color belongs to the data above the mean for each country
# Likewise, the data that is below the mean is denoted as blue color
sns.set(style="whitegrid")

plt.figure(figsize=(10,6))

sns.swarmplot(x='scored_signif',y='country',
              data=combined_drama_df[combined_drama_df['above_mean'] == 'Low'], legend=False,
              color='blue', alpha=0.6, label='Above Mean')

sns.swarmplot(x='scored_signif',y='country',hue='tot_watched', 
              data=combined_drama_df[combined_drama_df['above_mean'] == 'High'], legend=True,
              palette='flare',alpha=1.0, label='Below Mean')

sns.swarmplot(x='scored_signif',y='country',
              data=combined_outliers_df, legend=False,
              marker="x", linewidth=2.0, label='(Removed) Outliers')

plt.xlabel("Score Significance")
plt.ylabel("Country")

legend_handles = [
    mlines.Line2D([], [], color='red', label='Mean & Above', marker='o', markersize=4, linestyle='None'),
    mlines.Line2D([], [], color='blue', label='Below Mean', marker='o', markersize=4, linestyle='None'),
    mlines.Line2D([], [], color='black', label='(Remove) Outliers', marker='x', markersize=4, linestyle='None')
]

plt.legend(handles=legend_handles, loc='upper right', bbox_to_anchor=(1.0, 0.75), fontsize='small')

title_text = r'$\bf{Score}$' + ' ' + r'$\bf{Significance}$' + '\nDistribution by Country (from 2021-23)'
plt.title(title_text, fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

palette = sns.color_palette('Set2')

sns.kdeplot(
    data=combined_drama_df, x='scored_signif', hue='country',
    fill=True, common_norm=False,
    palette=palette, alpha=0.5, linewidth=0
)

country_means = combined_drama_df.groupby('country')['scored_signif'].mean()
for i, (country, mean) in enumerate(country_means.items()):
    plt.axvline(mean, color=palette[i], linestyle='--', label=f'Mean ({country}): {mean:.2f}')

plt.xlabel('Scored Significant')
plt.ylabel('Density')
plt.title('Distribution of Scored Significant by Country')
plt.legend()
plt.show()

# Hypothesis Testing:


In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis

df = combined_drama_df.copy()
korean_data = df[df['country'] == 'South Korea']['scored_signif']
thai_data = df[df['country'] == 'Thailand']['scored_signif']
japan_data = df[df['country'] == 'Japan']['scored_signif']


print("KOREA")
# Calculate skewness and kurtosis
skewness = skew(korean_data)
kurt = kurtosis(korean_data)

# Print the results
print(f"Skewness: {skewness:.2f}")
print(f"Kurtosis: {kurt:.2f}")

print("THAILAND")
skewness = skew(thai_data)
kurt = kurtosis(thai_data)

# Print the results
print(f"Skewness: {skewness:.2f}")
print(f"Kurtosis: {kurt:.2f}")


print("JAPAN")
skewness = skew(japan_data)
kurt = kurtosis(japan_data)

# Print the results
print(f"Skewness: {skewness:.2f}")
print(f"Kurtosis: {kurt:.2f}")

When we say that a dataset can still be considered approximately normal for many statistical purposes despite having some deviation in skewness and kurtosis, it means that the deviations are relatively small and may not have a significant impact on the validity of certain statistical analyses that assume normality.

1. **Benchmark or Range for Skewness:**
   - A skewness value of 0 indicates a perfectly symmetric distribution (no skew).
   - A positive skewness value indicates right skew (data stretched to the right).
   - A negative skewness value indicates left skew (data stretched to the left).
   - For many statistical analyses, skewness values between -1 and +1 are considered small and may not pose a problem. Beyond this range, particularly extreme skewness (e.g., greater than +2 or less than -2), may raise concerns about the assumption of normality.

2. **Benchmark or Range for Kurtosis:**
   - A kurtosis value of 3 indicates the kurtosis of a standard normal distribution (mesokurtic).
   - A positive kurtosis value (greater than 3) indicates heavy tails compared to a normal distribution (leptokurtic).
   - A negative kurtosis value (less than 3) indicates lighter tails compared to a normal distribution (platykurtic).
   - Similar to skewness, for many statistical analyses, kurtosis values within a certain range (e.g., -2 to +2) may be considered small deviations from normality and may not significantly affect results. Extreme kurtosis values (much greater than +2 or much less than -2) may warrant caution.

It's important to note that the interpretation of skewness and kurtosis should consider the specific context of your analysis and the statistical method being used. Some statistical tests and methods are more robust to deviations from normality than others. Additionally, the sample size can also influence the impact of deviations on statistical results.

If your analysis relies heavily on the assumption of normality (e.g., parametric tests like t-tests or ANOVA), it's a good practice to assess normality using both statistical tests (e.g., Shapiro-Wilk test) and visual methods (e.g., Q-Q plots) in addition to examining skewness and kurtosis. If deviations from normality are substantial, you may need to consider alternative non-parametric tests or transformations of the data.
- **In our case, the deviation isn't susbstantial; hence, we can still purposed hypothesis testing like ANOVA**



### Boostrapping

Bootstrapping should be used for hypothesis testing with skewed data because it provides more robust and reliable results by reducing the influence of outliers and non-normality, making it a valuable tool for assessing statistical significance in such scenarios.

Summary:

Bootstrapping mitigates skewness by:

1. Reducing the impact of outliers.
2. Creating replicated datasets.
3. Being non-parametric (no distribution assumptions).
4. Providing confidence intervals.
5. Facilitating hypothesis testing on skewed data.

In [ ]:
# Reference: https://www.datatipz.com/blog/hypothesis-testing-with-bootstrapping-python
# The method has been modified to fit with the data + Annotation and Additional Artistic Approach
from tqdm.auto import tqdm
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def get_bootstrap(
    data_column_1, 
    data_column_2, 
    iterations=15000, 
    statistic=np.mean, 
    sign_level=0.05,
    name_1=None,
    name_2=None
) -> None:
    
    boot_len = max(len(data_column_1), len(data_column_2))
    boot_data, samples_1_a, samples_2_a = [], [], []

    # Bootstrap sampling
    for _ in tqdm(range(iterations)):
        sample_1, sample_2 = data_column_1.sample(boot_len, replace=True).values, data_column_2.sample(boot_len, replace=True).values
        boot_data.append(statistic(sample_2 - sample_1))
        samples_1_a.append(np.mean(sample_1))
        samples_2_a.append(np.mean(sample_2))

    # Calculate quantiles
    quants = pd.DataFrame(boot_data).quantile([sign_level / 2, 1 - sign_level / 2])

    # Calculate p-values
    boot_mean, boot_std = np.mean(boot_data), np.std(boot_data)
    p_1 = norm.cdf(0, boot_mean, boot_std)
    p_2 = norm.cdf(0, -boot_mean, boot_std)
    p_value = min(p_1, p_2) * 2

    fig, axes = plt.subplots(2, 1, figsize=(10, 10), dpi=100)
    
    # Plot 1
    axes[0].grid(True, linestyle='-', linewidth=0.7, color='lightgrey')
    axes[0].set_facecolor('white')
    n, _, bars = axes[0].hist(boot_data, bins=50)
    for bar in bars:
        if bar.get_x() <= quants.iloc[0][0] or bar.get_x() >= quants.iloc[1][0]:
            bar.set_facecolor('palevioletred')
        else: 
            bar.set_facecolor('deepskyblue')
        bar.set_edgecolor('purple')

    axes[0].vlines(boot_mean, ymin=0, ymax=max(n), linestyle='-', color='orange', linewidth=4.0)   

    # Add annotations for p-value, boot_mean, H0, and H1
    axes[0].annotate(f"p = {np.round(p_value, 6)}", xy=(0.045, 0.9), xycoords="axes fraction", fontsize=12, color='red')
    # axes[0].annotate(f"Boot Mean = \n{np.round(boot_mean, 6)}", xy=(0.05, 0.75), xycoords="axes fraction", fontsize=12, color='black')
    axes[0].annotate("H0: The average \nscore significance \nis equal", xy=(0.045, 0.55), xycoords="axes fraction", fontsize=12, color='black')
    axes[0].annotate("H1: The average score\nsignificance is different", xy=(0.75, 0.6), xycoords="axes fraction", fontsize=12, color='black')

    axes[0].vlines(quants, ymin=0, ymax=max(n), linestyle='--', color='purple', linewidth=2.0)
    axes[0].annotate(f"{quants.iloc[0][0]:.3f}", xy=(quants.iloc[0][0], 0), xytext=(quants.iloc[0][0]+.001, max(n)/2))
    axes[0].annotate(f"{quants.iloc[1][0]:.3f}", xy=(quants.iloc[1][0], 0), xytext=(quants.iloc[1][0]+.001, max(n)/2))
    axes[0].set(xlabel='Bootstrap Data', ylabel='Frequency', title=f'Distribution of differences in {statistic.__name__}s\nBootstrap Mean Difference: {np.round(boot_mean, 6)}')
    
    # Plot 2
    axes[1].grid(True, linestyle='-', linewidth=0.7, color='lightgrey')
    axes[1].set_facecolor('white')
    axes[1].hist(samples_1_a, bins=50, color='deepskyblue', alpha=0.5, label=f"{name_1}'s {statistic.__name__} (Mean={np.round(np.mean(samples_1_a), 6)})")
    axes[1].hist(samples_2_a, bins=50, color='crimson', alpha=0.5, label=f"{name_2}'s {statistic.__name__} (Mean={np.round(np.mean(samples_2_a), 6)})")
    axes[1].set(ylabel='Frequency')
    axes[1].legend()
    plt.subplots_adjust(hspace=0.2)

    # Result
    conclusion = "reject" if p_value <= sign_level else "fail to reject"
    axes[1].annotate(f"Result: {conclusion} H0", xy=(0.40, 0.6), xycoords="axes fraction", fontsize=12, color='black')

df = combined_drama_df.copy()
korean_data = df[df['country'] == 'South Korea']['scored_signif']
thai_data = df[df['country'] == 'Thailand']['scored_signif']

# print(f"Korea: {np.mean(korean_data)}, Thai: {np.mean(thai_data)}")
get_bootstrap(thai_data, korean_data, name_1='Thailand', name_2='South Korea')


In [ ]:
df = combined_drama_df.copy()
korean_data = df[df['country'] == 'South Korea']['scored_signif']
jap_data = df[df['country'] == 'Japan']['scored_signif']

get_bootstrap(jap_data, korean_data, name_1='Japan', name_2='South Korea')

### Hypothese testing for South Korea vs. Thailand

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np

# South Korean Drama vs. Lakorn Thai

df = combined_drama_df.copy()
korean_data = df[df['country'] == 'South Korea']['scored_signif']
thai_data = df[df['country'] == 'Thailand']['scored_signif']

# Perform a two-sample t-test
t_statistic, p_value = stats.ttest_ind(korean_data, thai_data, alternative='greater')

# Set the significance level (alpha)
alpha = 0.05

# Create a range of x-values for the t-distribution curve
x = np.linspace(-5, 5, 1000)

# Create the t-distribution curve
t_distribution = stats.t.pdf(x, len(korean_data) + len(thai_data) - 2)

# Create a figure
plt.figure(figsize=(10, 6))

# Plot the t-distribution
plt.plot(x, t_distribution, label='t-Distribution', color='blue')

# Shade the region corresponding to the p-value
if t_statistic > 0:
    plt.fill_between(x, 0, t_distribution, where=(x >= t_statistic), color='red', alpha=0.5, label='p-Value')
else:
    plt.fill_between(x, 0, t_distribution, where=(x <= -t_statistic), color='red', alpha=0.5, label='p-Value')

# Add text annotations for the p-value and significance level
if t_statistic > 6:
    plt.text(3, 0.12, f'p = {p_value:.4f}', fontsize=12, color='red')
    plt.text(3, 0.15, f't-statistics = {t_statistic:.4f}', fontsize=12, color='black')
else:
    plt.text(t_statistic, 0.05, f'p = {p_value:.4f}', fontsize=12, color='red')
    
plt.axvline(
    x=stats.t.ppf(1 - alpha / 2, len(korean_data) + len(thai_data) - 2), 
    color='green', linestyle='--', label=f'Significance Level\n(alpha={alpha})',
    linewidth=2.5,
)

hypothesis_text = f"""Null Hypothesis (H0): The average score significance \nof Korean drama is equal to that of Lakorn Thai. \n
Alternative Hypothesis (H1): The average score significance \nof Korean drama is greater than that of Lakorn Thai.
"""
plt.text(6.0, 0.3, hypothesis_text, fontsize=12, color='gray', ha='left', va='top')

if p_value < alpha:
    analysis_result = f"Reject the null hypothesis: The average score significance \nof Korean drama is greater than that of Lakorn Thai."
else:
    analysis_reusult = f"Fail to reject the null hypothesis: There is not enough evidence to conclude \nthat the average score significance of Korean drama is greater than that of Lakorn Thai."

plt.text(6.0, 0.15, f"Hypothesis Testing Result:", color='blue', ha='left', va='top')
plt.text(6.0, 0.125, analysis_result, fontsize=12, color='black', ha='left', va='top')
    
# Set labels and title
plt.xlabel('t-Values')
plt.ylabel('Probability Density')
plt.title('t-Distribution\nSouth Korean Drama vs. Lakorn Thai')
plt.legend()
plt.grid(False)

plt.show()

### Hypothesis Testing for South Korea vs. Japan

In [ ]:
# South Korean vs Japanese Drama
df = combined_drama_df.copy()
korean_data = df[df['country'] == 'South Korea']['scored_signif']
thai_data = df[df['country'] == 'Japan']['scored_signif']

# Perform a two-sample t-test
t_statistic, p_value = stats.ttest_ind(korean_data, thai_data, alternative='greater')

# Set the significance level (alpha)
alpha = 0.05

# Create a range of x-values for the t-distribution curve
x = np.linspace(-5, 5, 1000)

# Create the t-distribution curve
t_distribution = stats.t.pdf(x, len(korean_data) + len(thai_data) - 2)

# Create a figure
plt.figure(figsize=(10, 6))

# Plot the t-distribution
plt.plot(x, t_distribution, label='t-Distribution', color='blue')

# Shade the region corresponding to the p-value
if t_statistic > 0:
    plt.fill_between(x, 0, t_distribution, where=(x >= t_statistic), color='red', alpha=0.5, label='p-Value')
else:
    plt.fill_between(x, 0, t_distribution, where=(x <= -t_statistic), color='red', alpha=0.5, label='p-Value')

# Add text annotations for the p-value and significance level
if t_statistic > 6:
    plt.text(3, 0.12, f'p = {p_value:.4f}', fontsize=12, color='red')
    plt.text(3, 0.15, f't-statistics = {t_statistic:.4f}', fontsize=12, color='black')
else:
    plt.text(t_statistic, 0.05, f'p = {p_value:.4f}', fontsize=12, color='red')
    
plt.axvline(
    x=stats.t.ppf(1 - alpha / 2, len(korean_data) + len(thai_data) - 2), 
    color='green', linestyle='--', label=f'Significance Level\n(alpha={alpha})',
    linewidth=2.5,
)

hypothesis_text = f"""Null Hypothesis (H0): The average score significance \nof Korean drama is equal to that of Japanese Drama. \n
Alternative Hypothesis (H1): The average score significance \nof Korean drama is greater than that of Japanese Drama.
"""
plt.text(6.0, 0.3, hypothesis_text, fontsize=12, color='gray', ha='left', va='top')

if p_value < alpha:
    analysis_result = f"Reject the null hypothesis: The average score significance \nof Korean drama is greater than that of Japanese Drama."
else:
    analysis_reusult = f"Fail to reject the null hypothesis: There is not enough evidence to conclude \nthat the average score significance of Korean drama is greater than that of Japanese Drama."

plt.text(6.0, 0.15, f"Hypothesis Testing Result:", color='blue', ha='left', va='top')
plt.text(6.0, 0.125, analysis_result, fontsize=12, color='black', ha='left', va='top')
    
# Set labels and title
plt.xlabel('t-Values')
plt.ylabel('Probability Density')
plt.title('t-Distribution\nSouth Korean Drama vs. Japanese Drama')
plt.legend()
plt.grid(False)

plt.show()

### ANOVA: Is there a significant difference in the means of the groups (South Kore, Thailand, and Japan)

In [ ]:
df = combined_drama_df.copy()
grouped_data = [df['scored_signif'][df['country'] == country] for country in df['country'].unique()]

f_statistic, p_value = stats.f_oneway(*grouped_data)

alpha = 0.05

x = np.linspace(0, 5, 1000)
f_distribution = stats.f.pdf(x, len(df['country'].unique()) - 1, len(df['scored_signif']) - len(df['country'].unique()))

plt.figure(figsize=(10, 6))
plt.plot(x, f_distribution, label='F-Distribution', color='blue')

plt.fill_between(x, 0, f_distribution, where=(x >= f_statistic), color='red', alpha=0.5, label='p-Value')

plt.text(3.5, 0.55, f'F-Statistic = {f_statistic:.4f}', fontsize=12, color='black')
plt.text(3.5, 0.5, f'p = {p_value:.4f}', fontsize=12, color='red')
plt.axvline(
    x=stats.f.ppf(1 - alpha, len(df['country'].unique()) - 1, 
    len(df['scored_signif']) - len(df['country'].unique())), 
    color='green', linestyle='--', label=f'Significance Level\n(alpha={alpha})'
)

hypothesis_text = f"""Null Hypothesis (H0): There is no sigificant difference \namong the means of scored significant better \nthree countries (μ1 = μ2 = μ3) \n
Althernative Hypothesis (H1): At least one population \nmean is different from others (μ1 ≠ μ2 ≠ μ3)
"""

plt.text(5.5, 0.7, hypothesis_text, fontsize=12, color='gray', ha='left', va='top')

if p_value < alpha:
    analysis_result = f"Reject the null hypothesis: There is a significant\ndifference in the means of at least two groups"
else:
    analysis_reusult = f"Fail to reject the null hypothesis: There is no significant\ndifference in the means of the groups"
    
plt.text(5.5, 0.33, f"Hypothesis Testing Result:", fontsize=12, color='blue', ha='left', va='top')
plt.text(5.5, 0.25, analysis_result, fontsize=12, color='black', ha='left', va='top')

plt.xlabel('F-Values')
plt.ylabel('Probability Density')
plt.title('F-Distribution:\nSouth Korea, Thai, and Japan')
plt.legend()
plt.grid(False)
plt.show()


In [ ]:
from collections import Counter 

genres_count_list = []

for genres in dkor_df_filtered['genres'].to_list():
    for genre in genres:
        genres_count_list.append(genre)

genre_df = pd.DataFrame.from_dict(Counter(genres_count_list),orient='index').rename(columns={0:'Count'})
genre_df.sort_values(by='Count',ascending = False,inplace = True)
genre_df = genre_df.reset_index().rename(columns={'index':'genre'})
genre_df.head(10)

In [ ]:
from wordcloud import WordCloud
from matplotlib.colors import LinearSegmentedColormap

genre_count_dict = dict(zip(genre_df['genre'], genre_df['Count']))

wordcloud = WordCloud(
    width=800, height=400, background_color='black', 
    contour_color = 'black', colormap = 'gray',
    normalize_plurals=True
).generate_from_frequencies(genre_count_dict)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Remove axis labels
plt.show()


------

# We might disregard this "Review" section since we couldn't get much of the information?

## Reviews

Reviews can give a more indept analysis on why these drama are so great. However, not every drama has the same number of comments. In this case, we will just do the analysis based on the existing reviews. 

For each review, the scoring can be seperated into mutliple sub-scoring; hence, each field provides more insight!

#### Preprocessing Task
We will preprocess that data. Firstly, we defined a function called `review_preprocess_pipeline` that extract useful information such as total number of episode the reviewer watched, which we can used to calculated the `watched_ratio`, This ratio will be used as a weighting factor to find weighted average rating score or `wrat_avg_score` reviewers gave to the drama.

#### Why should we consider using a weighted score in our analysis? 
To illustrate this, let's draw a parallel with shopping on an e-commerce website. Imagine you come across two identical products being sold by two different vendors. In most cases, you would likely opt to purchase from the vendor with a stronger reputation. This reputation can manifest in various ways: a higher number of followers, likes, or exceptionally high average review scores, such as a perfect 5.0 stars.

Now, let's apply a similar line of thinking to our analysis. When it comes to reviews of a particular drama, we value input from individuals with greater credibility. This credibility comes from those who have already watched the entire drama, as they can provide a more comprehensive and informed perspective on it. Therefore, we may choose to assign more weight to reviews from viewers with a proven track record of completing the drama, just as we would preferentially buy from a trusted vendor when shopping online.

### Probability Problem: 
Given that we picked one Lakorn Thai randomly, what is the probability that the Lakorn is top rated and belongs to Horror, Romance or  

In [ ]:
def review_preprocess_pipeline(
    df: pd.DataFrame,
    country: str
) -> pd.DataFrame:
    
    # Filtered for review that is valid
    df = df[df['title'].isin(valid_drama_dict[country])].iloc[:,1:]
    
    # Create Additional Features into each Dataset as a Key to determine the associated Country of the Drama being commented
    df['country'] = country
    
    # Removed the people that watched more than the number of episodes?
    
    # Extract Number of Epsiode watched and Total Episode Number
    df['ep_watched'] = df['ep_watched'].fillna('0 of 0 episodes seen')
    df['tot_watched'] = df['ep_watched'].apply(lambda x: int(str(x).split(' ')[0]))
    df['tot_ep'] = df['ep_watched'].apply(lambda x: int(str(x).split(' ')[2]))
    
    # Feature Engineering
    df['ep_watched_ratio'] = df['tot_watched'] / df['tot_ep']
    df['ep_watched_ratio'] = df['ep_watched_ratio'].fillna(0.0).apply(lambda x: round(x,3))
    
    df['avg_score'] = df[['story','acting_cast','music','rewatch_value']].mean(axis=1)
    df['wrat_avg_score'] = df['avg_score'] * df['ep_watched_ratio']
    
    # Remove Features that wouldn't be part of the analysis 
    df = df.drop(columns=['ep_watched', 'text'], axis=1)
    
    return df

In [ ]:
rtha_df = pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/tha_user_reviews.csv')
rkor_df = pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/kor_user_reviews.csv')
rjap_df = pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/jap_user_reviews.csv')

rtha_df_f = review_preprocess_pipeline(rtha_df,'Thailand')
rkor_df_f = review_preprocess_pipeline(rkor_df, 'South Korea')
rjap_df_f = review_preprocess_pipeline(rjap_df, 'Japan')

rtha_df_f.shape, rkor_df_f.shape, rjap_df_f.shape

In [ ]:
rtha_outliers, rtha_df_filtered = process_outliers_df_pipeline(rtha_df_f, 'wrat_avg_score')
rkor_outliers, rkor_df_filtered = process_outliers_df_pipeline(rkor_df_f, 'wrat_avg_score')
rjap_outliers, rjap_df_filtered = process_outliers_df_pipeline(rjap_df_f, 'wrat_avg_score')

In [ ]:
rtha_df_filtered.head()

In [ ]:
def draw_boxplots_with_annotations(
    dataframe: pd.DataFrame,
    country: str = '',
) -> None:
    sns.reset_orig()
    
    num_columns = len(dataframe.columns)
    num_rows = (num_columns + 2) // 3
    
    fig, axes = plt.subplots(nrows=num_rows, ncols=3, figsize=(16, 6 * num_rows // 2))
    axes = axes.flatten()

    for i, col in enumerate(dataframe.columns):
        # Create a boxplot for the current column in the corresponding subplot
        ax = axes[i]
        ax.boxplot(dataframe[col], vert=False, sym='b.', 
                   whis=1.5, patch_artist=True, boxprops=dict(facecolor='lightblue'))

        # Outlier Calculation using IQR Method
        q1 = np.percentile(dataframe[col], 25)
        q3 = np.percentile(dataframe[col], 75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = [x for x in dataframe[col] if x < lower_bound or x > upper_bound]
        num_outliers = len(outliers)
        
        mean = np.mean(dataframe[col])
        std_dev = np.std(dataframe[col])
        annotation = f"Mean: {mean:.2f}\nStd Dev: {std_dev:.2f}\nOutliers: {num_outliers}"
        ax.text(0.62, 0.80, annotation, transform=ax.transAxes, fontsize=12,
                verticalalignment='center')
        
        min_score = np.min(dataframe[col])
        max_score = np.max(dataframe[col])
        minmax_annotation = f"Min: {min_score:.2f}\nMax: {max_score:.2f}"
        ax.text(0.62, 0.25, minmax_annotation, transform=ax.transAxes, fontsize=12,
               verticalalignment='center')

        ax.set_title(col)
    
    big_title = f"{country} Boxplots"
    fig.suptitle(big_title, fontsize=16, y=1.02)
    
    for j in range(num_columns, len(axes)):
        fig.delaxes(axes[j])
        
    plt.tight_layout()
    plt.show()

# features = [
#     'story', 'acting_cast', 'music', 'rewatch_value', 'overall',
#     'n_helpful', 'tot_watched', 'tot_ep', 'ep_watched_ratio',
#     'avg_score', 'wrat_avg_score'
# ]

features = ['ep_watched_ratio','overall','wrat_avg_score']

draw_boxplots_with_annotations(rtha_df_filtered[features], 'Thailand')

draw_boxplots_with_annotations(rkor_df_filtered[features], 'South Korea')

draw_boxplots_with_annotations(rjap_df_filtered[features], 'Japan')

Do we have to manage the outliers based on ep_watched_ratio, n_helpful, etc.?

Whether to remove the outliers before or after performing a feature engineering like weighted averge score depends on the specific golas and requirement of this analysis. Upon observation of the fields related to score, we notice some outliers. However, this is something we would like to capture. The minimum and maximum values for these fields are between 1 and 10; hence, there is no hidden error in the data extraction or scoring process on the website. They are valuable insights even if they were outliers; henceforth, we won't remove it for this moment. 


Address other features and how does outlier in those feature effect the final weighted score


In [ ]:
print("Removed based the the Episode Watched Ratio: ")
_, rtha_df_f2 = process_outliers_df_pipeline(rtha_df_filtered, 'ep_watched_ratio')
_, rkor_df_f2 = process_outliers_df_pipeline(rkor_df_filtered, 'ep_watched_ratio')
_, rjap_df_f2 = process_outliers_df_pipeline(rjap_df_filtered, 'ep_watched_ratio')

print("Removed based on Weigthed Ratio Average Score: ")
_, rtha_df_f2 = process_outliers_df_pipeline(rtha_df_f2, 'wrat_avg_score')
_, rkor_df_f2 = process_outliers_df_pipeline(rtha_df_f2, 'wrat_avg_score')
_, rjap_df_f2 = process_outliers_df_pipeline(rtha_df_f2, 'wrat_avg_score')



features = ['ep_watched_ratio','overall','wrat_avg_score']

draw_boxplots_with_annotations(rtha_df_f2[features], 'Thailand')

draw_boxplots_with_annotations(rkor_df_f2[features], 'South Korea')

draw_boxplots_with_annotations(rjap_df_f2[features], 'Japan')

In [ ]:
rtha_df_f2[rtha_df_f2['title'] == '10 Years Ticket']

In [ ]:
# Testing the aggregate function
agg_rtha_df = (
    rtha_df_f2
    .reset_index()
    .groupby(['title'])
    .agg({'wrat_avg_score':'mean', 'index':'count'})
    .rename(columns={'wrat_avg_score': 'avr_wrat_score', 'index':'num_reviews'})
    .reset_index()
)

agg_rtha_df.head()

-----

## Actors Processing

In [ ]:
atha_df = pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/tha_tha_actors.csv')
akor_df = pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/kor_kor_actors.csv')
ajap_df = pd.read_csv('/kaggle/input/thai-and-japanese-drama-vs-korean-drama-dominance/jap_jap_actors.csv')

atha_df.head()

In [ ]:
# Create a function to check uniqueness of the actor name
def check_actor_name_uniqueness(df: pd.DataFrame) -> None:
    
    actor_dict = {}
    
    for index, row in df.iterrows():
        actor_id, actor_name = row['actor_id'], row['actor_name']

        if actor_name not in actor_dict.keys():
            actor_dict[actor_name] = {
                'id_list': []
            }

        if actor_id not in actor_dict[actor_name]['id_list']:
            actor_dict[actor_name]['id_list'].append(actor_id)

    for key, val in actor_dict.items():
        actor_dict[key]['nunique'] = len(actor_dict[key]['id_list'])

        if actor_dict[key]['nunique'] != 1:
            print("Actor name isn't unique!")
            return 
    
    print("Actor name is UNIQUE!")
    return 

# Check the uniqueness of the actor name. Need to use them as key later-on
check_actor_name_uniqueness(atha_df)
check_actor_name_uniqueness(akor_df)
check_actor_name_uniqueness(ajap_df)

Write a function to merge the information of actor and drama (might have to be the new dataframe created from the processed reviews?) to get the information regarding the drama performance that each actor acted in. If have to use the processed data from review, then move code blocks related to actors to be after the user reviews sections

In [ ]:
df = atha_df.copy()

drama_df = dtha_df_filtered[['drama_name','genres','tags','country','content_rt','scored_signif']].copy()
country = 'Thailand'

df = df.drop(columns=['actor_id','character_name'],axis=1)
df = df[df['drama_name'].isin(valid_drama_dict[country])]

merged_df = df.merge(drama_df, on='drama_name', how='left')

merged_df.head()


In [ ]:
merged_df.groupby(['actor_name']).count()[['drama_name']].sort_values(['drama_name'], ascending=False)

In [ ]:
merged_df[merged_df['actor_name'] == 'Pond Naravit Lertratkosum']

In [ ]:
# Popular Actor with Main role based on Average Significane Score of the Drama they perforemd in
# Create Pipeline...
actor_score_df = merged_df.groupby(['actor_name','role']).agg({'scored_signif':'mean'}).rename(columns={'scored_signif':'avg_score'}).reset_index()
actor_score_df[actor_score_df['role'] == 'Main Role'].sort_values(['avg_score'], ascending=False)

In [ ]:
'''
Dictionary Architectures:

actor_name
    - role
        - genres
        - tags
        - content_rt
    - appearance

val: scored_signig
'''
# Convert this dictionary into dataframe
actor_meta_db = {}

# Focus Only on Main Role and Support Role
sdf = merged_df[merged_df['role'].isin(['Main Role','Support Role'])]

for index, row in sdf.iterrows():
    
    name, role, score = row['actor_name'], row['role'], row['scored_signif']
    
    if name not in actor_meta_db.keys():
        actor_meta_db[name] = {
            'Main Role': {},
            'Support Role': {}
        }

    for genre in row['genres']:
        if genre != 'nan' and genre not in actor_meta_db[name][role].keys():
            actor_meta_db[name][role][genre] = {
                'score_db': [],
                'count': 0,
            }
            
        if genre in actor_meta_db[name][role].keys():
            actor_meta_db[name][role][genre]['score_db'].append(score)
            
            actor_meta_db[name][role][genre]['count'] += 1

# import json 
# print(json.dumps(actor_meta_db, indent=4))
            
for key, item in actor_meta_db.items():

    for role in ['Main Role', 'Support Role']:
        if item[role].keys():
            for genre, (score_list_db, counter) in item[role].items():
                average_score = np.mean(item[role][genre][score_list_db])

                actor_meta_db[key][role][genre][score_list_db] = round(average_score,3)
    
# import json 
# print(json.dumps(actor_meta_db, indent=4))

# Initialize empty lists to store data
actor_names = []
roles = []
genres = []
average_scores = []
num_appearance = []

# Iterate through the dictionary to extract data
for actor_name, roles_data in actor_meta_db.items():
    for role, genres_data in roles_data.items():
        for genre, data in genres_data.items():
            actor_names.append(actor_name)
            roles.append(role)
            genres.append(genre)
            average_scores.append(data['score_db'])
            num_appearance.append(data['count'])

# Create a Pandas DataFrame
df = pd.DataFrame({
    'actor_name': actor_names,
    'role': roles,
    'genres': genres,
    'average_score': average_scores,
    'num_appearance': num_appearance,
})

# Print the DataFrame
df.head()

In [ ]:
'''
Dictionary Architectures:

actor_name
    - role
        - genres
        - tags
        - content_rt
    - appearance

val: scored_signig
'''
# Convert this dictionary into dataframe
actor_meta_db = {}

# Focus Only on Main Role and Support Role
sdf = merged_df[merged_df['role'].isin(['Main Role','Support Role'])]

for index, row in sdf.iterrows():
    
    name, role, score = row['actor_name'], row['role'], row['scored_signif']
    
    if name not in actor_meta_db.keys():
        actor_meta_db[name] = {
            'Main Role': {},
            'Support Role': {}
        }

    for tag in row['tags']:
        if tag != "(Vote or add tags)" and genre not in actor_meta_db[name][role].keys():
            actor_meta_db[name][role][tag] = {
                'score_db': [],
                'count': 0,
            }
            
        if tag in actor_meta_db[name][role].keys():
            actor_meta_db[name][role][tag]['score_db'].append(score)
            
            actor_meta_db[name][role][tag]['count'] += 1

# import json 
# print(json.dumps(actor_meta_db, indent=4))
            
for key, item in actor_meta_db.items():

    for role in ['Main Role', 'Support Role']:
        if item[role].keys():
            for tag, (score_list_db, counter) in item[role].items():
                average_score = np.mean(item[role][tag][score_list_db])

                actor_meta_db[key][role][tag][score_list_db] = round(average_score,3)
    
# import json 
# print(json.dumps(actor_meta_db, indent=4))

# Initialize empty lists to store data
actor_names = []
roles = []
tags = []
average_scores = []
num_appearance = []

# Iterate through the dictionary to extract data
for actor_name, roles_data in actor_meta_db.items():
    for role, genres_data in roles_data.items():
        for genre, data in genres_data.items():
            actor_names.append(actor_name)
            roles.append(role)
            tags.append(genre)
            average_scores.append(data['score_db'])
            num_appearance.append(data['count'])

# Create a Pandas DataFrame
df = pd.DataFrame({
    'actor_name': actor_names,
    'role': roles,
    'tags': tags,
    'average_score': average_scores,
    'num_appearance': num_appearance,
})

# Print the DataFrame
df.head()

In [ ]:
df[(df['actor_name'] == 'Mai Davika Hoorne') & (df['tags'] == 'Prolonged Nudity')]

In [ ]:
df[['actor_name','tags','average_score']].groupby(['tags']).agg({'average_score':'mean'}).reset_index().sort_values('average_score',ascending=False).head(10)

In [ ]:
'''
Dictionary Architectures:

actor_name
    - role
        - genres
        - tags
        - content_rt
    - appearance

val: scored_signig
'''
# For Tags
actor_meta_db = {}

# Focus Only on Main Role and Support Role
sdf = merged_df[merged_df['role'].isin(['Main Role','Support Role'])]

for index, row in sdf.iterrows():
    
    name, role, score = row['actor_name'], row['role'], row['scored_signif']
    
    if name not in actor_meta_db.keys():
        actor_meta_db[name] = {
            'Main Role': {},
            'Support Role': {}
        }
    
    for tag in row['tags']:
        if tag != "(Vote or add tags)" and tag not in actor_meta_db[name][role].keys():
            actor_meta_db[name][role][tag] = []

        if tag in actor_meta_db[name][role].keys():
            actor_meta_db[name][role][tag].append(score)

for key, item in actor_meta_db.items():

    for role in ['Main Role', 'Support Role']:
        if item[role].keys():
            for tag, score_list in item[role].items():
                average_score = np.mean(score_list)

                actor_meta_db[key][role][tag] = round(average_score,3)
    
    
import json
# print(json.dumps(actor_meta_db, indent=4))